In [1]:
import requests
from config import req_headers, base_url, focus_league, local_host, local_name, local_port, local_user
from creds import local_db_password
import psycopg2
import time

In [2]:
conn = psycopg2.connect(dbname=local_name, user =local_user, host=local_host, password=local_db_password, port =local_port)
cur = conn.cursor()

In [3]:
endpoint = 'players'
url = base_url + endpoint
season = 2023

In [4]:
player_list = []
fetch_teams_sql = f"""select * from(
select player_id from(
select 
	fixture_id,
	team_id,
	cast(players -> 'player' -> 'id' as int) as player_id,
	coalesce(cast(players -> 'statistics' -> 0 -> 'games' ->> 'minutes' as int), 0) as minutes,
	coalesce(cast(players -> 'statistics' -> 0 -> 'games' ->> 'rating' as float), 0) as rating,
	cast(players -> 'statistics' -> 0 -> 'games' ->> 'captain' as bool) as captain,
	cast(players -> 'statistics' -> 0 -> 'games' ->> 'substitute' as bool) as substitute,
	coalesce(cast(players -> 'statistics' -> 0 ->> 'offsides' as int), 0) as offsides,
	coalesce(cast(players -> 'statistics' -> 0 -> 'shots' ->> 'total' as int), 0) as shots_total,
	coalesce(cast(players -> 'statistics' -> 0 -> 'shots' ->> 'on' as int), 0) as shots_on,
	coalesce(cast(players -> 'statistics' -> 0 -> 'goals' ->> 'total' as int), 0) as goals_total,
	coalesce(cast(players -> 'statistics' -> 0 -> 'goals' ->> 'conceded' as int), 0) as goals_conceded,
	coalesce(cast(players -> 'statistics' -> 0 -> 'goals' ->> 'assists' as int), 0) as assists,
	coalesce(cast(players -> 'statistics' -> 0 -> 'goals' ->> 'saves' as int), 0) as saves,
	coalesce(cast(players -> 'statistics' -> 0 -> 'passes' ->> 'total' as int), 0) as passes_total,
	coalesce(cast(players -> 'statistics' -> 0 -> 'passes' ->> 'key' as int), 0) as passes_key,
	coalesce(cast(players -> 'statistics' -> 0 -> 'passes' ->> 'accuracy' as int), 0) as passes_accuracy,
	coalesce(cast(players -> 'statistics' -> 0 -> 'tackles' ->> 'total' as int), 0) as tackles_total,
	coalesce(cast(players -> 'statistics' -> 0 -> 'tackles' ->> 'blocks' as int), 0) as blocks,
	coalesce(cast(players -> 'statistics' -> 0 -> 'tackles' ->> 'interceptions' as int), 0) as interceptions,
	coalesce(cast(players -> 'statistics' -> 0 -> 'dribbles' ->> 'past' as int), 0) as dribbles_past,
	coalesce(cast(players -> 'statistics' -> 0 -> 'dribbles' ->> 'success' as int), 0) as dribbles_success,
	coalesce(cast(players -> 'statistics' -> 0 -> 'dribbles' ->> 'attempts' as int), 0) as dribbles_attempts,
	coalesce(cast(players -> 'statistics' -> 0 -> 'fouls' ->> 'drawn' as int), 0) as fouls_drawn,
	coalesce(cast(players -> 'statistics' -> 0 -> 'fouls' ->> 'committed' as int), 0) as fouls_committed,
	coalesce(cast(players -> 'statistics' -> 0 -> 'cards' ->> 'yellow' as int), 0) as cards_yellow,
	coalesce(cast(players -> 'statistics' -> 0 -> 'cards' ->> 'red' as int), 0) as cards_red,
	coalesce(cast(players -> 'statistics' -> 0 -> 'penalty' ->> 'won' as int), 0) as penalty_won,
	coalesce(cast(players -> 'statistics' -> 0 -> 'penalty' ->> 'committed' as int), 0) as penalty_committed,
	coalesce(cast(players -> 'statistics' -> 0 -> 'penalty' ->> 'scored' as int), 0) as penalty_scored,
	coalesce(cast(players -> 'statistics' -> 0 -> 'penalty' ->> 'missed' as int), 0) as penalty_missed,
	coalesce(cast(players -> 'statistics' -> 0 -> 'penalty' ->> 'saved' as int), 0) as penalty_saved,
	coalesce(cast(players -> 'statistics' -> 0 -> 'duals' ->> 'won' as int), 0) as duals_won,
	coalesce(cast(players -> 'statistics' -> 0 -> 'duals' ->> 'total' as int), 0) as duals_total
from (
	select 
		fixture_id,
		cast(teams -> 'team' ->> 'id' as int) as team_id,
		jsonb_array_elements(teams::jsonb->'players') players
	from (
		select 
			cast(response_data::jsonb -> 'parameters' ->> 'fixture' as int) as fixture_id,
			jsonb_array_elements(response_data::jsonb->'response') teams
		from ffl_staging.query_data
		where status = 'loaded' and query_scope = 'fixtures/players') a 
		) b ) c
group by player_id ) d
where player_id not in (select id from ffl.players)"""
cur.execute(fetch_teams_sql)
for i in cur.fetchall():
    player_list.append(i[0])

In [5]:
player_insert_sql = """
insert into ffl.players (
    id,
    current_team_id,
    season,
    position,
    name_first,
    name_last,
    name_easy,
    height,
    weight,
    date_of_birth
)
values
(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
on conflict do nothing
"""

In [6]:
def parser(data=None):
    player_id = int(data['player']['id'])
    team_id = int(data['statistics'][0]['team']['id'])
    player_season = season
    position = data['statistics'][0]['games']['position']
    name_first = data['player']['firstname']
    name_last = data['player']['lastname']
    name_easy = data['player']['name']
    height = data['player']['height']
    weight = data['player']['weight']
    date_of_birth = data['player']['birth']['date']

    return player_id, team_id, player_season, position, name_first, name_last, name_easy, height, weight, date_of_birth

In [7]:
def fetch_response(params=None, page = None):
    if page == None:
        req = requests.get(url, params=params, headers=req_headers)

        return req.json()
    else:
        params['page'] = page
        req = requests.get(url, params=params, headers=req_headers)

        return req.json()

def load_responses(response_data=None):
    for player in response_data['response']:
        cur.execute(player_insert_sql, (parser(player)))
    # conn.commit()

In [8]:
for player in player_list:
    call_params = {
        # 'league':focus_league,
        'season':season,
        'id':player}

    response = fetch_response(call_params)
    load_responses(response)
    conn.commit()

ForeignKeyViolation: insert or update on table "players" violates foreign key constraint "fk_player_team"
DETAIL:  Key (current_team_id)=(7747) is not present in table "teams".


In [ ]:
response